In [1]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import LLMChain, APIChain
from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.schema import Document
from langchain.tools import BaseTool
from langchain.agents import ZeroShotAgent, AgentExecutor, AgentType, initialize_agent, Tool, load_tools
from langchain.agents.agent_toolkits.openapi import planner
from langchain.requests import TextRequestsWrapper
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.api import open_meteo_docs
import datetime
import requests
import pandas as pd
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
import numpy as np

In [2]:
# Load up your LLM
llm = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_base='https://openai-nois-intern.openai.azure.com/',
    openai_api_version="2023-03-15-preview",
    deployment_name='gpt-35-turbo',
    openai_api_key='400568d9a16740b88aff437480544a39',
    temperature=0.5,
    max_tokens=450
)

llm2 = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_base='https://openai-nois-intern.openai.azure.com/',
    openai_api_version="2023-03-15-preview",
    deployment_name='gpt-35-turbo',
    openai_api_key='400568d9a16740b88aff437480544a39',
    temperature=0.0,
    max_tokens=600,
    stop=['\n', '<|im_end|>', '<|im_sep|>']
)

llm3 = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_base='https://openai-nois-intern.openai.azure.com/',
    openai_api_version="2023-03-15-preview",
    deployment_name='gpt-35-turbo-16k',
    openai_api_key='400568d9a16740b88aff437480544a39',
    temperature=0.0,
    max_tokens=600
)

history = []
history_entry = 1

def get_history_as_txt():
    txt = ""
    hist = history[::-1]
    
    for i in hist[:3]:
        txt += f"\n<|im_start|>user\n{i['user']}\n<|im_end|>\n"
        txt += f"<|im_start|>assistant\n{i['AI']}\n<|im_end|>"
        
    return txt

def add_to_history(user_msg, ai_msg):
    history.append({'user': user_msg, 'AI': ai_msg})
    
def save_history_to_file():
    global history_entry
    file = open(f"chatbot-cases/chat-{history_entry}.txt", 'w', encoding='utf-8')
    file.write(get_history_as_txt())
    
    file.close()
    
    history_entry += 1
    
def doc_to_str(doc):
    string = ""
    for i in doc:
        string += i.page_content
        
    return string

# print(llm2)
doc = open_meteo_docs.OPEN_METEO_DOCS
# print(llm2.get_num_tokens(doc))

WARNING! stop is not default parameter.
                    stop was transferred to model_kwargs.
                    Please confirm that stop is what you intended.


In [8]:
url = "https://hrm-nois-fake.azurewebsites.net/"
email = 'bao.ho@nois.vn'
dtime = datetime.datetime
date = dtime.now().strftime("%Y-%m-%d")
print(date)

def get_users(query: str = None):
    return requests.get(url + '/api/User').json()['data']

def delAll(ID: str):
    lst = []
    get_reply = requests.get(url + f"/api/LeaveApplication/{ID}").json()
    for i in get_reply['data']:
        lst.append(i['id'])
        
    for i in lst:
        del_reply = requests.delete(url + f"/api/LeaveApplication/{i}")
        print(del_reply.text)

def get_user_by_email(query: str = None):
    user_email = email
    if '@' in query and query != user_email:
        return f"Access denied, only the data of the current user {user_email} can be accessed."
        
    response = requests.get(url + f'/api/User/me?email={user_email}')
    
    if response.status_code == 200:
        return_text = "User info:\n"
        user_data = response.json()['data']
        
        for i in user_data.items():
            return_text += f"- {i[0]}: {i[1]}\n"
            if i[0] == "maxDayOff":
                return_text += f"- remainingOffDays: {user_data['maxDayOff'] - user_data['offDay']}\n"
            
        return return_text
    
    return f'Error: {response.status_code}'

def output_to_user(query: str = None):
#     print(query)
    return f'Message "{query}" sent to user.'

def get_userName(query: str = None):
    user_email = email
    response = requests.get(url + f'/api/User/me?email={user_email}')
    if response.status_code == 200:
        return response.json()['data']['name']
    
    return f'Error: {response.status_code}'

def get_userId(query: str = None):
    user_email = email
    response = requests.get(url + f'/api/User/me?email={user_email}')
    if response.status_code == 200:
        return response.json()['data']['id']
    
    return f'Error: {response.status_code}'

def get_leave_applications(query: str = None):
    userId = get_userId(email)
    
    applis = requests.get(url + f'/api/LeaveApplication/{userId}').json()['data']
    
#     applis = [i for i in applis if i['reviewStatusName'] != "Đồng ý"]
    
    string = ""
    counter = 1
    for i in applis:
        string += f"Leave application {counter}:\n"
        string += f"ID: {i['id']}\n"
        string += f"Start date: {i['fromDate']}\n"
        string += f"End date: {i['toDate']}\n"
        string += f"Number of day(s) off: {i['numberDayOff']}\n\n"
        
        counter += 1
        
    return string

def delete_leave_applications(target: str = "Default"):
    applis = requests.get(url + f'/api/LeaveApplication/{get_userId(email)}').json()['data']
    
    appli = [i for i in applis if i['id'] == target]
    
    string = ""
    for i in appli:
        string += f"ID: {i['id']}\n"
        string += f"Start date: {i['fromDate']}\n"
        string += f"End date: {i['toDate']}\n"
        string += f"Number of day(s) off: {i['numberDayOff']}\n\n"
    
    user_confirm = input(string + "Are you sure you wnat to delete this application? Type 1 for yes, 0 for no.\n")
    
    while user_confirm != '1' and user_confirm != '0':
        user_confirm = input("Invalid input. Type 1 for yes, 0 for no.\n")
        
    if user_confirm == '0':
        return "User cancelled the leave application deletion process."
        
    return requests.delete(url + f'/api/LeaveApplication/{target}').json()['message']

def get_user_by_id(ID: str):
    reply = requests.get(url + f'/api/User/me?id={ID}')
    if response.status_code != 200:
        return f'Error: {response.status_code}'
    
    if not reply.json()['data']:
        return "Wrong ID, recheck your input."
    
    return reply.json()['data']

def check_manager_name(name):
    response = requests.get(url + '/api/User/manager-users').json()['data']
    
    for data in response:
        if data['fullName'].lower() == name.lower(): 
            return data['id']
        
    return -1
def check_manager_id(managerId):
    response = requests.get(url + '/api/User/manager-users').json()['data']
    
    for data in response:
        if data['id'] == int(managerId): 
            return True
        
    return False
    
def post_method(user_id, manager_id, start_date, end_date):
    start_dtime = dtime.strptime(start_date, "%Y-%m-%d")
    end_dtime = dtime.strptime(end_date, '%Y-%m-%d')
    
    num_days = 0
    period = 0
    
    if end_dtime == start_dtime:
        period = input("Which period do you want to leave? Type only the number respectively:\n0. All day\n1. Only the morning\n2. Only the afternoon\n\n")
        
        while period != "0" and period != "1" and period != "2":
            period = input("Invalid input. Please following the instruction above.\n")
            
        num_days = 0.5
            
        if period == "0":
            num_days = 1
        
    else:
        num_days = int(np.busday_count(start_date, end_date)) + 1
        
    string = f'''Leave application details:
    - UserId: {user_id}
    - ManagerId: {manager_id}
    - Start date: {start_date}
    - End date: {end_date}
    - Number of day(s) off: {num_days}
Is this information correct? Type 1 to submit, type 0 if you want to tell the bot to edit the form.\n'''
    
    user_confirm = input(string)
    
    while user_confirm != "1" and user_confirm != "0":
        user_confirm = input("Invalid input. Type 1 to submit, type 0 if you want to tell the bot to edit the form.\n")
    
    if user_confirm.strip() != "1":
        user_edit = input("Tell the bot what you want to edit in the form here:\n")
        
        return user_edit
        
    response = requests.post('https://hrm-nois-fake.azurewebsites.net/api/LeaveApplication/Submit', json = {
        "userId": user_id,
        "reviewUserId": manager_id,
        "relatedUserId": "string",
        "fromDate": start_date,
        "toDate": end_date,
        "leaveApplicationTypeId": 0,
        "leaveApplicationNote": "string",
        "periodType": 0,
        "numberOffDay": num_days
    })
    
    return response.json()['message']
    
lst = []
def submitLeaveApplication(args: str):
    global lst
    lst = args.split(', ')
    
    if len(lst) != 5 and len(lst) != 4:
        return "Incorrect number of parameters, this function has 4 required parameters(user's id, manager's id, start date and end date) and 1 optional parameter(note)."
    
    if '-' not in lst[0]:
        return "Incorrect ID. You can find the correct ID by using the HRM get by email tool."
    
    if check_manager_name(lst[1]) == -1 and check_manager_id(lst[1]) == False:
        return "Ask the user for the correct manager's name. The user either didn't give you a name or gave you the wrong name."
    
    if '-' not in lst[2]:
        return "Ask the user when they want to start their leave. Infer the date based on the user's answer."
    
    if dtime.strptime(lst[2], "%Y-%m-%d") < dtime.strptime(date, "%Y-%m-%d"):
        return "Start date cannot be earlier than current date, ask the user for another date."
    
    if dtime.strptime(lst[2], "%Y-%m-%d").weekday() in [5, 6]:
        return "Start date cannot be on the weekend, ask the user for another date."
    
    if '-' not in lst[3]:
        return "Ask the user when they want to end their leave. Infer the date based on the user's answer."
    
    if dtime.strptime(lst[3], "%Y-%m-%d") < dtime.strptime(lst[2], "%Y-%m-%d"):
        return "End date cannot be earlier than start date, ask the user for another date."
    
    if dtime.strptime(lst[3], "%Y-%m-%d").weekday() in [5, 6]:
        return "End date cannot be on the weekend, ask the user for another date."
    
    print("\nUserId: ", lst[0])
    print("ReviewerId: ", lst[1])
    print("Start date: ", lst[2])
    print("End date: ", lst[3])
    print('Note: ', lst[4])
    
    reply = post_method(lst[0], int(lst[1]), lst[2], lst[3], lst[4])
    
    return reply
#     return '{"message": "OK"}'

def end_convo(query: str = None):
    return """Output Final Answer: Conversation ended after this message."""

def datetime_calc(query: str):
    try:
        return eval(query)
    except Exception:
        return exec(query)

2023-07-19


In [9]:
def nothing(query: str = None):
    return ""

temp1_backup = f'''

Examples: 
Question: What is my date of birth?
Thought: The user chatting with me has the email {email}.
Thought: find the data of user by user's email.
Action: HRM get user data
Action Input: {email}
Observation: ...
Thought: I now know the final answer
Final Answer: Your date of birth is ...

Question: What is hung.bui@nois.vn's bank account number?
Thought: The user chatting with me has the email {email}.
Thought: the question is not about the current user, so I can't answer the question.
Action: human
Action Input: I'm sorry but I don't have access to that information, can I help you with anything else?
Observation: No
Thought: I now know the final answer
Final Answer: 

'''

tool1 = [
    Tool(
        name='HRM get user data',
        func=get_user_by_email,
        description='useful for getting data of the current user, data include fullName, birthday, bankAccountNumber, etc. No input necessary.'
    ),
    
    #     Tool(
#         name='Output to user',
#         func=output_to_user,
#         description='useful for sending messages to the user as they cannot see your thought, actions, etc. This tool takes in your message and sends it to the user.'
#     ),
    
#     Tool(
#         name='End conversation',
#         func=end_convo,
#         description='useful for ending the conversation if the user cancels the task or you cannot complete the task.'
#     ),
    
#     Tool(
#         name='Do nothing',
#         func=nothing,
#         description='useful for filling in Action: after Thought: if the task can\'t be done or the task is cancelled.'
#     ),
]

human = load_tools(['human'])
tool1.extend(human)

prefix1 = f"""You are an intelligent assistant helping user find his/her information in HRM system by using tool. 
HRM system is a system which store employees's data of a company, but you only able to answer questions about the current user.
The user chatting with you is {get_userName(email)} and he/she own the email: {email}. 
If the question is not about the current user:
- No search anything.
- Must not use data of the current user {email} to answer to the question about other users.
You have access to the following tools:"""

temp1 = f'''

Examples: 
Question: What is hung.bui@nois.vn's bank account number?
Thought: The user chatting with you own the email {email}.
Thought: The question is not about the current user, do not reponse anything.
Final Answer: Sorry, due to security policy, I can not answer your question.
Finished chain.

Question: I want to know Bao Ho's mobile phone number?
Thought: The user chatting with you own the email {email}.
Thought: The question is not about the current user, do not reponse anything.
Final Answer: Sorry, due to security policy, I can not answer your question.
Finished chain.

Question: What is my date of birth?
Thought: The user chatting with me has the email {email}.
Thought: find the data of user by user's email.
Action: HRM get user data
Action Input: {email}
Observation: 12/5/2000
Thought: The answer is founded.
Final Answer: Your date of birth is 12/5/2000
Finished chain.
'''

suffix1 = '''
Begin!

Conversation history:
{context}

Question: {input}
{agent_scratchpad} 
'''

suffix1 = temp1 + suffix1

prompt1 = ZeroShotAgent.create_prompt(
    tool1,
    prefix=prefix1,
    suffix=suffix1,
    input_variables=["input", "context", "agent_scratchpad"],
)
llm_chain1 = LLMChain(llm=llm, prompt=prompt1)

agent1 = ZeroShotAgent(llm_chain=llm_chain1, tools=tool1, verbose=True, stop=["\nObservation:", "<|im_end|>", "<|im_sep|>"])
history1 = ConversationBufferWindowMemory(k=3, memory_key="context", human_prefix='User', ai_prefix='Assistant')
agent_chain1 = AgentExecutor.from_agent_and_tools(
    agent=agent1,
    tools=tool1, 
    verbose=True,
    handle_parsing_errors="True",
    memory=history1
)

In [10]:
# print(prompt1.template)
history1.clear()
agent_chain1.run(input="How many off-days do I have left?")



> Entering new  chain...
Thought: The user chatting with me has the email bao.ho@nois.vn.
Thought: Find the data of user by user's email.
Action: HRM get user data
Action Input: bao.ho@nois.vn
Observation: User info:
- id: 7cc99c43-1951-4bc1-a1c9-ca3487b1d6f3
- name: Bao Ho
- email: bao.ho@nois.vn
- fullName: HỒ NGUYỄN NGỌC BẢO
- jobTitle: Lập Trình Viên
- mobilePhone: 
- birthday: 2002-04-30
- offDay: 3
- yearOffDay: 3
- maxDayOff: 12
- remainingOffDays: 9
- remainDayOffLastYear: 0
- offDayUseRamainDayOffLastYear: 0
- gender: 1
- personalEmail: honguyenngocbao@gmail.com
- avatar: 
- acceptOfferDate: 
- quitJobDate: 
- bonusDayOff: 0
- managerId: 139
- managerFullName: LÝ MINH QUÂN
- departmentId: 0
- departmentName: 
- groupUserId: 
- groupUserName: 
- hasLeft: 
- identityCard: 
- dateOfIssue: 
- issuedBy: 
- placeOfTemporaryResidence: 
- placeOfResidence: 
- relativeName: 
- relationship: 
- relativeMobilePhone: 
- hasBankAccount: 
- bankName: 
- bankBranch: 
- bankAccountNumber: 


'You have 9 off-days left.'

In [15]:
tool2 = [   
    Tool(
        name='HRM get userId',
        func=get_userId,
        description='useful for getting the user\'s id by user\'s email. No need to input'
    ),
    
    Tool(
        name='HRM get by name',
        func=check_manager_name,
        description='useful for getting the manager\'s id by manager\'s name. Input is a manager\'s name.'
    ),
    
    Tool(
        name='HRM submit leave',
        func=submitLeaveApplication,
        description=f'''useful for submitting a leave applications for current user.
        
Input of this tool need include 4 required parameters and 1 optional paramter, they are concatenated into a string separated by a comma and space, which are: 
1. user's id (required): you get by the tool HRM get userId. 
2. the id of manager who approve user's application (required): firstly, ask user the name of the manager and then use the tool HRM get by name to find manager's id.
3. start date (required): ask the user when he/she want to start their leave, then automatically convert start date to format Year-Month-Day and save it (if nesessary).
4. end date (required): ask the user when he/she want to end their leave, then automatically convert start date to format Year-Month-Day and save it (if nesessary). The end date can be same as the start date.
5. note (optional): ask the user what he/she want to give a note to his/her manager. Set this fill into a null string if the user do not want to note.
Important: Always show the input informations(manager's name, start date, end date, note) from user and ask the user to confirm before submitting leave application.'''
    ),
#     Tool(
#         name='End conversation',
#         func=end_convo,
#         description="useful for ending a chat conversation at the user's request or when you have accomplished your task."
#     )
    
    Tool(
        name='Calculate time',
        func=datetime_calc,
        description='useful for calculating dates. Input is a python code utilizing the datetime library.'
    ),
    
    Tool(
        name='HRM get applications',
        func=get_leave_applications,
        description='useful for getting leave applications of the current user, mostly for deletion tasks. Input is a "None" string.'
    ),
    
    Tool(
        name='HRM delete leave application',
        func=delete_leave_applications,
        description='useful for deleting a specific leave application. Input is the ID of the leave application. Always run the HRM get applications first and ask the user which application they want to delete.'
    ),
]

human = load_tools(['human'])
tool2.extend(human)

f'''If you cannot answer by using only 1 tool, try using other provided tools.
If you cannot get any results from tools, say you don't know.
Do not use any other sources other than the ones you obtain from tools.
Suppose the current date is {date} (Year-Month-Day).'''

date2 = dtime.strftime(dtime.today(), "%A %Y-%m-%d")
print(date2)
# date2 = "Friday, 2023-12-29"

prefix2 = f"""You are an intelligent assistant helping user submit or delete leave applications through the HRM system using tools. 
Only do one task (submit or delete leave application) in one chain and must finish chain immediately after completing task.
The user chatting with you has the email: {email}. 
Suppose the current date is {date2} (Weekday Year-Month-Day).
You have access to the following tools:"""

temp2_backup = f'''

Example:  
Question: I'd like to submit leave application.
Thought: The user chatting with you own the email {email}.
Thought: Find the user's id by {email}.
Action: HRM get userId
Action Input: {email}
Observation: {get_userId(email)}

Thought: After having user's id, ask the user for the manager's name.
Action: human
Action Input: What is the name of your manager?
Observation: lý minh quân
Thought: find the manager's id by manager's name.
Action: HRM get by name
Action Input: lý minh quân
Observation: 139

Thought: Ask user the date of starting leave (according the format Year-Month-Day)?
Action: human
Action Inpunt: Let me know when do you want to start your leave?
Observation: 2023-07-17

Thought: Ask user the date of ending leave (according the format Year-Month-Day)?
Action: human
Action Input: How about the date of ending leave?
Observation: 2023-07-18

Thought: Got all details for submitting.
Action: HRM submit leave
Action Input: {get_userId(email)}, 139, 2023-07-17, 2023-07-18
Observation: Successfully submitted.
Thought: Leave application is submitted.
Final Answer: Leave application is submitted.

'''

temp2 = f'''

Example:  
Question: I'd like to submit leave application.
Thought: The user chatting with you own the email {email}.
Thought: Find the user's id by {email}.
Action: HRM get userId
Action Input: {email}
Observation: {get_userId(email)}

Thought: After having user's id, ask the user for the manager's name.
Action: human
Action Input: What is the name of your manager?
Observation: lý minh quân
Thought: find the manager's id by manager's name.
Action: HRM get by name
Action Input: lý minh quân
Observation: 139

Thought: Ask user the date of starting leave.
Action: human
Action Input: Let me know when do you want to start your leave? (According to format year-month-day)
Observation: 2023-07-17

Thought: Ask user the date of ending leave.
Action: human
Action Input: How about the date of ending leave? (According to format year-month-day)
Observation: 2023-07-18

Thought: Ask the user if they want to note anything.
Action: Human
Action Input: Do you want to note anything?
Observation: Please allow me to study at school.

Thought: Got all details for submitting.
Action: HRM submit leave
Action Input: {get_userId(email)}, 139, 2023-07-17, 2023-07-18, 'Please allow me to study at school'
Observation: OK
Thought: Leave application is submitted.
Final Answer: Leave application is submitted.

'''

suffix2 = """Begin!

{context}
Question: {input}
{agent_scratchpad} 
"""

suffix2 = temp2 + suffix2

prompt2 = ZeroShotAgent.create_prompt(
    tool2,
    prefix=prefix2,
    suffix=suffix2,
    input_variables=["input", "context", "agent_scratchpad"],
)


llm_chain2 = LLMChain(llm=llm, prompt=prompt2)
history2 = ConversationBufferWindowMemory(k=3, memory_key="context", human_prefix='User', ai_prefix='Assistant')

agent2 = ZeroShotAgent(llm_chain=llm_chain2, tools=tool2, verbose=True, stop=["\nObservation:", "<|im_end|>", "<|im_sep|>"])
agent_chain2 = AgentExecutor.from_agent_and_tools(
    agent=agent2,
    tools=tool2, 
    verbose=True,
    handle_parsing_errors="True",
    memory=history2
)

Wednesday 2023-07-19


In [16]:
# print(prompt2.template)

In [19]:
history.clear()
answer = agent_chain2.run(input="I'd like to submit a leave application.")



> Entering new  chain...
Thought: The user wants to submit a leave application.
Thought: Find the user's id by bao.ho@nois.vn.
Action: HRM get userId
Action Input: bao.ho@nois.vn
Observation: 7cc99c43-1951-4bc1-a1c9-ca3487b1d6f3
Thought:After having user's id, ask the user for the manager's name.
Observation: Invalid Format: Missing 'Action:' after 'Thought:'
Thought:Question: What is the name of your manager?
Thought: I need to ask the user for the name of their manager.
Action: human
Action Input: What is the name of your manager?

What is the name of your manager?


KeyboardInterrupt: Interrupted by user

In [ ]:
print(lst)

In [25]:
delAll("7cc99c43-1951-4bc1-a1c9-ca3487b1d6f3")

{"message":"OK","data":"success"}
{"message":"OK","data":"success"}


In [26]:
lst = [('2023-07-17', '2023-07-18'), ('2023-07-19', '2023-07-19'), ('2023-07-20', '2023-07-21')]

for start_date, end_date in lst:
    start_dtime = dtime.strptime(start_date, "%Y-%m-%d")
    end_dtime = dtime.strptime(end_date, '%Y-%m-%d')

    num_days = 0

    if end_dtime == start_dtime:
        num_days = 0.5
    else:

        num_days = int(np.busday_count(start_date, end_date)) + 1

    response = requests.post('https://hrm-nois-fake.azurewebsites.net/api/LeaveApplication/Submit', json = {
        "userId": '7cc99c43-1951-4bc1-a1c9-ca3487b1d6f3',
        "reviewUserId": '139',
        "fromDate": start_date,
        "toDate": end_date,
        "leaveApplicationTypeId": 1,
        "leaveApplicationNote": "string",
        "periodType": 0,
        "numberOffDay": num_days
    })

    print(response.text)

{"message":"OK"}
{"message":"OK"}
{"message":"OK"}


In [19]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

def count_tokens(s):
    return len(enc.encode(s))

print(count_tokens(temp2))

337


In [27]:
print(get_leave_applications())

Leave application 1:
ID: 26a4ecf3-dd29-4ea0-9b73-b2417fb8786a
Start date: 2023-07-19T00:00:00
End date: 2023-07-19T00:00:00
Number of day(s) off: 0.5

Leave application 2:
ID: 71119b92-6ca7-4c65-b43f-418d1f795c42
Start date: 2023-07-17T00:00:00
End date: 2023-07-18T00:00:00
Number of day(s) off: 2

Leave application 3:
ID: df095f5e-2650-4d16-8362-dae5cf7ef6ad
Start date: 2023-07-20T00:00:00
End date: 2023-07-21T00:00:00
Number of day(s) off: 2




2